# Transformar fitxer en format Excel a format CSV

OBSERVACIÓ: Si teniu dubtes del codi, contactar amb rdr-contacte@csuc.cat

## OBJECTIU DE L'SCRIPT

L'objectiu principal d'aquest script és transformar les fulles d'un fitxer excel en diferents fitxers CSV

In [ ]:
# @title Install necessary packages
!pip install pandas openpyxl

# Import required libraries
import shutil
import os
import pandas as pd
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [ ]:
# @title Upload the excel file
import os
import pandas as pd
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import shutil

xls_file = None  # Declare xls_file variable outside the function
directory_name = None  # Declare directory_name as a global variable

def detect_and_save_tables(sheet_name, df):
    # Initialize variables
    tables = []
    current_table = []

    # Iterate over rows
    for index, row in df.iterrows():
        # Check if the row is completely empty
        if row.isnull().all():
            if current_table:
                tables.append(current_table.copy())
                current_table = []
        else:
            current_table.append(row)

    # Append the last table
    if current_table:
        tables.append(current_table.copy())

    # Convert each table to DataFrame and save as CSV
    csv_files = []
    for i, table in enumerate(tables):
        table_df = pd.DataFrame(table)
        csv_file = f"{sheet_name}_{i+1}.csv"
        table_df.to_csv(csv_file, index=False, header=False)  # Disable header
        csv_files.append(csv_file)

    return csv_files

def process_excel(change):
    global directory_name  # Define directory_name as a global variable

    clear_output(wait=True)

    # Check if any files are uploaded
    if not change.new:
        print("Please upload a file.")
        return

    # Get the uploaded file
    uploaded_file = next(iter(change.new.values()))
    excel_content = uploaded_file['content']

    global xls_file  # Access the global variable
    # Read the Excel file
    try:
        xls_file = pd.ExcelFile(excel_content)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    # Define directory_name
    directory_name = os.path.splitext(uploaded_file['metadata']['name'])[0]

    # Create a directory to store CSV files
    os.makedirs(directory_name, exist_ok=True)

    # Create buttons for user input
    yes_button = widgets.Button(description="Yes", button_style="success")
    no_button = widgets.Button(description="No", button_style="danger")
    buttons_box = widgets.HBox([yes_button, no_button])
    display(HTML("<p style='font-size:14px;'><b>Separate tables in each sheet into different CSV files?</b></p>"))
    display(buttons_box)

    def on_yes_button_clicked(b):
        clear_output(wait=True)
        separate_tables = 'yes'
        process_sheet(separate_tables)

    def on_no_button_clicked(b):
        clear_output(wait=True)
        separate_tables = 'no'
        process_sheet(separate_tables)

    yes_button.on_click(on_yes_button_clicked)
    no_button.on_click(on_no_button_clicked)

def process_sheet(separate_tables):
    # Iterate over each sheet
    for sheet_name in xls_file.sheet_names:
        df = pd.read_excel(xls_file, sheet_name=sheet_name, header=None)

        # Determine whether to separate tables or not
        if separate_tables == 'yes':
            csv_files = detect_and_save_tables(sheet_name, df)
        else:
            csv_file = f"{sheet_name}.csv"
            df.to_csv(csv_file, index=False, header=False)  # Disable header
            csv_files = [csv_file]

        # Move CSV files to directory
        for csv_file in csv_files:
            new_csv_file = os.path.join(directory_name, csv_file)
            if os.path.exists(new_csv_file):
                os.remove(new_csv_file)  # Remove existing file
            shutil.move(csv_file, new_csv_file)

    # Create a zip file of the directory
    zip_file = shutil.make_archive(directory_name, 'zip', directory_name)

    # Trigger the download of the zip file
    files.download(zip_file)

# Displaying file upload message
display(HTML("<p style='font-size:14px;'><b>Please upload your Excel file.</b></p>"))

# Creating upload button
upload_button = widgets.FileUpload()
upload_button.observe(process_excel, names='value')

# Displaying the upload button
display(upload_button)
